In [1]:
import os
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
DATA_DIR = "data"
SCORES_DIR = os.path.join(DATA_DIR, "scores")

# Start

In [107]:
html_pages = os.listdir('html_pages2/html2')
html_pages_path = []
for a in html_pages:
    a = f"html_pages2/html2/{a}"
    html_pages_path.append(a)
    

In [109]:
print(len(html_pages_path))
print(html_pages_path[0])

# RECORDS FOR PAST 5 SEASONS


6269
html_pages2/html2/201810160BOS.html


In [19]:
def parse_html(box_score):
    with open(box_score, encoding='utf8') as f:
        html = f.read()
        
        
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [20]:
def read_date(soup):
    nav = soup.select('#bottom_nav_container')[0]
    date = str(nav.a.u)
    date = date[3:15]
    
    return date

In [21]:
def read_away_team(soup):
    nav = soup.select('#bottom_nav_container')[0]
    away_team = str(nav.li.a).split('/')[2]
    return away_team

In [24]:
def read_home_team(soup):
    nav = soup.select('#bottom_nav_container')[0]
    home_team = nav.select('a')
    home_team = str(home_team[2]).split("/")[2]
    return home_team

In [25]:
# taking out index_col=0 from pd.read_html(,)
def read_stats(soup, team, level):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{level}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df
                      

In [84]:
def read_line_score(soup):
    nav = soup.select('#bottom_nav_container')[0]
    away = nav.li
    away = str(away).split('/')[2]
    
    home = nav.find_all('li')[1]
    home = str(home).split('/')[2]
    
    line_df = {'home_team': home, 'away_team': away}
    line_df = pd.DataFrame(data=line_df, index=[0])
    
    return line_df

In [ ]:
# --- TEST FUNCTION ---

games = []
base_cols = None
for box_score in test1:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)

    team1 = list(line_score["home_team"])
    team2 = list(line_score["away_team"])
    teams = team1 + team2  

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        
        summaries.append(totals)
        
        
    #summaries[1].index += "_opp"
    summary = pd.concat(summaries, axis=1).T
    summary = summary.reset_index(drop=True)
    home_series = summary.iloc[0]
    away_series = summary.iloc[1]
    totals_row = home_series.append(away_series)
    totals_row_df = pd.DataFrame(totals_row).T
    # home away and date
    totals_row_df['home'] = read_home_team(soup)
    totals_row_df['away'] = read_away_team(soup)
    totals_row_df['date'] = read_date(soup)
                                       
    #totals_row_df['won'] = totals_row_df['PTS'] > totals_row_df['PTS_opp']
    games.append(totals_row_df)


In [100]:
# first scrape errored out at 4855

# new list of paths will begin at 4855

#html_pages_path = html_pages_path[4855:]

# link number 4855 is screwed up, only showing CSS code, going to skip for now


In [ ]:
# --- ADVANCED STATS ---

games = []
base_cols = None
for box_score in html_pages_path:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)

    team1 = list(line_score["home_team"])
    team2 = list(line_score["away_team"])
    teams = team1 + team2  

    summaries = []
    for team in teams:
        #basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")
        
        #totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        
        summaries.append(advanced.iloc[-1,:])
        
        
    summary = pd.concat(summaries, axis=1).T
    summary = summary.reset_index(drop=True)
    home_series = summary.iloc[0]
    home_series.index = home_series.index.str.lower()
    away_series = summary.iloc[1]
    away_series.index = away_series.index.str.lower() + "_opp"
    totals_row = home_series.append(away_series)
    totals_row_df = pd.DataFrame(totals_row).T
    # home away and date
    totals_row_df['home'] = read_home_team(soup)
    totals_row_df['away'] = read_away_team(soup)
    totals_row_df['date'] = read_date(soup)
                                       
    #totals_row_df['won'] = totals_row_df['PTS'] > totals_row_df['PTS_opp']
    games.append(totals_row_df)


In [119]:
#html_pages_path = html_pages_path[4855:]

In [121]:
#html_pages_path = html_pages_path[1:]

In [ ]:
# --- BASIC STATS ---
games = []
base_cols = None
for box_score in html_pages_path:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)

    team1 = list(line_score["home_team"])
    team2 = list(line_score["away_team"])
    teams = team1 + team2  

    summaries = []
    for team in teams:
        basic = read_stats(soup, team, "basic")
        #advanced = read_stats(soup, team, "advanced")
        
        #totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        #basic_total = basic.iloc[-1,:]
        summaries.append(basic.iloc[-1,:])
        
        
    #summaries[1].index += "_opp"
    summary = pd.concat(summaries, axis=1).T
    summary = summary.reset_index(drop=True)
    home_series = summary.iloc[0]
    home_series.index = home_series.index.str.lower()
    away_series = summary.iloc[1]
    away_series.index = away_series.index.str.lower() + "_opp"
    totals_row = home_series.append(away_series)
    totals_row_df = pd.DataFrame(totals_row).T
    # home away and date
    totals_row_df['home'] = read_home_team(soup)
    totals_row_df['away'] = read_away_team(soup)
    totals_row_df['date'] = read_date(soup)
                                       
    #totals_row_df['won'] = totals_row_df['PTS'] > totals_row_df['PTS_opp']
    games.append(totals_row_df)


In [106]:
# TWO DATASETS WITH 1 ROW MISSING (4855)
    #games_df.to_csv('data_advanced1.csv')
    #games_df.to_csv('data_advanced2.csv')

In [123]:
games_df = pd.concat(games, ignore_index=False)
games_df


,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,...,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,+/-_opp,home,away,date
0,240.0,45.0,84.0,0.536,9.0,28.0,0.321,16.0,24.0,0.667,...,26.0,9.0,2.0,15.0,21.0,108.0,NaN,BRK,CLE,"Apr 12, 2022"
0,240.0,36.0,83.0,0.434,14.0,35.0,0.400,23.0,37.0,0.622,...,19.0,5.0,5.0,17.0,28.0,104.0,NaN,MIN,LAC,"Apr 12, 2022"
0,240.0,49.0,94.0,0.521,16.0,32.0,0.500,18.0,25.0,0.720,...,22.0,4.0,4.0,14.0,24.0,103.0,NaN,ATL,CHO,"Apr 13, 2022"
0,240.0,44.0,81.0,0.543,8.0,20.0,0.400,17.0,23.0,0.739,...,19.0,4.0,3.0,6.0,20.0,103.0,NaN,NOP,SAS,"Apr 13, 2022"
0,240.0,37.0,81.0,0.457,13.0,36.0,0.361,14.0,17.0,0.824,...,15.0,5.0,3.0,13.0,19.0,107.0,NaN,CLE,ATL,"Apr 15, 2022"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,240.0,40.0,79.0,0.506,8.0,27.0,0.296,16.0,20.0,0.800,...,26.0,5.0,4.0,8.0,15.0,93.0,NaN,DEN,MIA,"Jun 1, 2023"
0,240.0,39.0,75.0,0.520,11.0,28.0,0.393,19.0,22.0,0.864,...,28.0,5.0,4.0,11.0,22.0,111.0,NaN,DEN,MIA,"Jun 4, 2023"
0,240.0,34.0,92.0,0.370,11.0,35.0,0.314,15.0,19.0,0.789,...,28.0,3.0,5.0,13.0,18.0,109.0,NaN,MIA,DEN,"Jun 7, 2023"
0,240.0,35.0,78.0,0.449,8.0,25.0,0.320,17.0,20.0,0.850,...,26.0,11.0,7.0,6.0,18.0,108.0,NaN,MIA,DEN,"Jun 9, 2023"
